# Convolutional Neural Network

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="3QZVAyiL3YQKQum9mQ0h")
project = rf.workspace("saishashank524-gmail-com").project("rescue")
dataset = project.version(2).download("folder")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.9/69.9 kB 688.5 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.3/158.3 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.5/77.5 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.9.0.80
    Uninstalling opencv-python-headless-4.9.0.80:
      Successfully uninstalled opencv-python-headless-4.9.0.80
  Attempting uninstall: idna
    Found existing installation: idna 3.6
    Uninstalling idna-3.6:
      Successfully uninstalled idna-3.6
  Attempting uninstall: cycler
    Found existing installation: cycle

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Rescue-2 in folder:: 100%|██████████| 9819/9819 [00:04<00:00, 2111.57it/s]


### Importing the libraries

In [2]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [80]:
tf.__version__

'2.15.0'

## Part 1 - Data Preprocessing

### Preprocessing the Training set

In [83]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
training_set = train_datagen.flow_from_directory('dataset/train',
                                                 target_size = (64, 64),
                                                 batch_size = 32,
                                                 class_mode = 'binary')

Found 6019 images belonging to 2 classes.


### Preprocessing the Test set

In [84]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

Found 491 images belonging to 2 classes.


## Part 2 - Building the CNN

### Initialising the CNN

In [94]:
cnn = tf.keras.models.Sequential()

### Step 1 - Convolution

In [95]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu', input_shape=[64, 64, 3]))

### Step 2 - Pooling

In [96]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Adding a second convolutional layer

In [97]:
cnn.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

### Step 3 - Flattening

In [98]:
cnn.add(tf.keras.layers.Flatten())

### Step 4 - Full Connection

In [99]:
for i in range(5):
    cnn.add(tf.keras.layers.Dense(units=128, activation='relu'))

### Step 5 - Output Layer

In [100]:
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the CNN

### Compiling the CNN

In [101]:
cnn.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

### Training the CNN on the Training set and evaluating it on the Test set

In [102]:
cnn.fit(x = training_set, validation_data = test_set, epochs = 5)

Epoch 1/5
189/189 [==============================] - 28s 130ms/step - loss: 0.2696 - accuracy: 0.8988 - val_loss: 0.1987 - val_accuracy: 0.9063
Epoch 2/5
189/189 [==============================] - 26s 139ms/step - loss: 0.1862 - accuracy: 0.9209 - val_loss: 0.2097 - val_accuracy: 0.9287
Epoch 3/5
189/189 [==============================] - 25s 133ms/step - loss: 0.1663 - accuracy: 0.9264 - val_loss: 0.1659 - val_accuracy: 0.9389
Epoch 4/5
189/189 [==============================] - 24s 128ms/step - loss: 0.1623 - accuracy: 0.9319 - val_loss: 0.1909 - val_accuracy: 0.9389
Epoch 5/5
189/189 [==============================] - 25s 134ms/step - loss: 0.1516 - accuracy: 0.9367 - val_loss: 0.2116 - val_accuracy: 0.9124


## Part 4 - Making a single prediction

In [107]:
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('dataset/valid/normal/normal_image0007_jpg.rf.8358aebccd05faf6956df07bb9fb16a5.jpg', target_size = (64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = cnn.predict(test_image)
training_set.class_indices
# if result[0][0] == 1:
#   prediction = 'dog'
# else:
#   prediction = 'cat'

1/1 [==============================] - 0s 21ms/step


{'collapsed_building': 0, 'normal': 1}

In [108]:
print(result)

[[1.]]


In [110]:
cnn.save('cnn')

In [114]:
!zip -r ccn_weights.zip cnn/

  adding: cnn/ (stored 0%)
  adding: cnn/variables/ (stored 0%)
  adding: cnn/variables/variables.index (deflated 68%)
  adding: cnn/variables/variables.data-00000-of-00001 (deflated 19%)
  adding: cnn/saved_model.pb (deflated 88%)
  adding: cnn/assets/ (stored 0%)
  adding: cnn/fingerprint.pb (stored 0%)
  adding: cnn/keras_metadata.pb (deflated 92%)


# Inference

In [ ]:
!unzip cnn.zip

In [1]:
import tensorflow as tf
import numpy as np
from keras.preprocessing import image

# Load the saved model
model = tf.keras.models.load_model('cnn/')

# Load and preprocess the test image
test_image = image.load_img('./frame3323.jpg', target_size=(64, 64))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)

# Perform inference
result = model.predict(test_image)

# Assuming `training_set` is the object containing class indices
print("Class Indices:", model.get_config()["class_indices"])
print("Inference Result:", result)

AttributeError: module 'keras.preprocessing.image' has no attribute 'load_img'